<p><b><center><span style="font-size:26px;" </span> <span style="font-family=Times new roman;"</span> Understanding different Mean Encodings schemes & Implementing them in Python</center></b></p>

<p><center><span style="font-size:16px;"</span> By Abdul Muqtadir()</center></p>


# 1. Introduction
Mean Encoding
1. In this tutorial we are going to Mean encode "Embarked" column. Using four different encoding schemes.
    * K-Fold Validation
    * Leave one out 
    * Smoothing
    * Expanding mean
2. Then we are going to find corelation in between every encoding scheme and target.

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

# Reading Dataset
Train_Data = pd.read_csv("../input/simple-dataset-for-easy-practice-for-beginners/BrandEDA.csv")

In [2]:
# About Dataset

#Train_Data.shape
#Train_Data.info()
Train_Data.dtypes
#Train_Data.nunique()
#Train_Data["Item ID"].unique()

Train_Data.head()


,Brand,Item ID,Ram(Gb),StoraGe(Gb),Price(Rs),Processor speed(GHz),Touch,Color,Weight,DisplAy size(Inch)
0,Apple,32,16.0,256.0,315000,2.2,No,Space grey,1.83,15.4
1,Apple,4,8.0,512.0,232000,3.2,No,Dark Black,1.29,13.3
2,Apple,32,16.0,1024.0,382000,2.3,No,Silver,2.00,16.0
3,Apple,4,8.0,NaN,195000,2.3,No,Space grey,1.37,13.3
4,Apple,4,4.0,128.0,92400,1.8,No,Silver,1.35,11.6


In [3]:
#Drop multiple columns (Just to make dataset bit easier for demonstration)
#Train_Data.drop(Train_Data.iloc[:, 14:26], axis=1, inplace=True)

In [4]:
Train_Data.sort_values(['Item ID'],inplace=True)

In [5]:
Train_Data["Brand"].replace({"Apple":"0", "Dell":"1"}, inplace = True)

In [6]:
Train_Data.head()
IID_GMean = Train_Data['Item ID'].mean()

# Mean Encoding with K-Fold Validation
***Steps***
1. **Import K-Fold with K=5 and shuffle = False**
2. **Split Train data in to Tr_d and Val_d**
3. **Iterate through K folds (If K=x This iteration will repeat x times)**
    * Train and Validation -(Assign)- train and validation index
    * Find Target mean for Train Folds groupedby(Categorical feature under operation)
    * New Feature in Validation Fold -(Assign)- Validation fold (Categorical feature under operation) .map(Target mean of Train folds)
    * Dataset(Index val) -(Assign)- Validation fold
4. **Fill null values with global mean in new feature of Dataset**

In [7]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=False)

for index_train, index_valid in kf.split(Train_Data):
    X_tr, X_val = Train_Data.iloc[index_train], Train_Data.iloc[index_valid]
    
    # target coding of valid dataset depends on train dataset
    X_tr_group = X_tr.groupby('Item ID')['Brand'] 
    X_val['item_target_enc'] = X_val['Item ID'].map(X_tr_group.mean())
    
    # copy target encoding back to all_data
    Train_Data.iloc[index_valid] = X_val
    

#Train_Data['item_target_enc'].fillna(IID_GMean, inplace=True)
Train_Data.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Brand,Item ID,Ram(Gb),StoraGe(Gb),Price(Rs),Processor speed(GHz),Touch,Color,Weight,DisplAy size(Inch)
9,0,4,16.0,NaN,382000,2.6,No,Silver,2.00,16.0
1,0,4,8.0,512.0,232000,3.2,No,Dark Black,1.29,13.3
3,0,4,8.0,NaN,195000,2.3,No,Space grey,1.37,13.3
4,0,4,4.0,128.0,92400,1.8,No,Silver,1.35,11.6
5,0,4,8.0,256.0,60000,2.6,No,Silver,1.00,13.3
